# Avaliação Few-Shot com Aprendizado Episódico

Este notebook implementa uma avaliação few-shot usando aprendizado episódico com redes de protótipos e um modelo Vision Transformer (DeiT) pré-treinado.

In [ ]:
import torch
import importlib
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
importlib.reload(__import__('helpers'))
from helpers import save_fewshot_results
import timm
from collections import defaultdict
import random
import torch.nn.functional as F
import numpy as np

## 1. Configuração do Experimento

In [ ]:
CONFIG = {
    'n_way': 3,
    'n_shot': 5,
    'n_query': 15,
    'n_episodes': 200,
    'model_name': 'deit_base_distilled_patch16_224',
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

## 2. Preparação dos Dados

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],   
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
dataset = datasets.ImageFolder(
    root=OUT_PATH,
    transform=transform
)

class_to_indices = defaultdict(list)
for idx, (_, label) in enumerate(dataset):
    class_to_indices[label].append(idx)


## 3. Funções Auxiliares para Episódios

In [ ]:
def create_episode(
    class_to_indices,
    n_way=5,
    n_shot=10,
    n_query=15
):
    classes = random.sample(list(class_to_indices.keys()), n_way)

    support_idx = []
    query_idx = []

    for c in classes:
        indices = random.sample(
            class_to_indices[c],
            n_shot + n_query
        )
        support_idx += indices[:n_shot]
        query_idx   += indices[n_shot:]

    return support_idx, query_idx, classes


In [ ]:
def get_episode_loaders(dataset, support_idx, query_idx):
    support_loader = DataLoader(
        Subset(dataset, support_idx),
        batch_size=len(support_idx),
        shuffle=False
    )

    query_loader = DataLoader(
        Subset(dataset, query_idx),
        batch_size=len(query_idx),
        shuffle=False
    )

    return support_loader, query_loader


In [ ]:
def extract_features_vit(loader, model):
    features = []
    labels = []

    model.eval()
    with torch.no_grad():
        for imgs, lbls in loader:
            imgs = imgs.to(next(model.parameters()).device)
            feats = model(imgs)
            features.append(feats)
            labels.append(lbls)

    return torch.cat(features), torch.cat(labels)

## 4. Implementação da Rede de Protótipos

In [ ]:
def evaluate_episode(
    model,
    dataset,
    class_to_indices,
    device,
    n_way=5,
    n_shot=10,
    n_query=15
):
    support_idx, query_idx, classes = create_episode(
        class_to_indices, n_way, n_shot, n_query
    )

    support_loader, query_loader = get_episode_loaders(
        dataset, support_idx, query_idx
    )

    support_features, support_labels = extract_features_vit(
        support_loader, model
    )
    query_features, query_labels = extract_features_vit(
        query_loader, model
    )

    original_query_labels = query_labels.cpu().numpy()

    support_features = F.normalize(support_features, p=2, dim=1)
    query_features   = F.normalize(query_features, p=2, dim=1)

    label_map = {c: i for i, c in enumerate(classes)}
    support_labels = torch.tensor(
        [label_map[int(l)] for l in support_labels],
        device=device
    )
    query_labels_remapped = torch.tensor(
        [label_map[int(l)] for l in query_labels],
        device=device
    )

    prototypes = torch.zeros(n_way, support_features.size(1)).to(device)
    for i in range(n_way):
        prototypes[i] = support_features[support_labels == i].mean(0)
    
    prototypes = F.normalize(prototypes, p=2, dim=1)

    sims = torch.mm(query_features, prototypes.t())
    preds_remapped = sims.argmax(dim=1)

    acc = (preds_remapped == query_labels_remapped).float().mean().item()
    
    original_preds = np.array([classes[int(p)] for p in preds_remapped.cpu().numpy()])
    
    return acc, preds_remapped.cpu().numpy(), query_labels_remapped.cpu().numpy(), original_preds, original_query_labels

## 5. Carregamento do Modelo

In [ ]:
model = timm.create_model(
    CONFIG['model_name'],
    pretrained=True,
    num_classes=0
)

model = model.to(CONFIG['device'])
model.eval()


## 6. Avaliação Episódica

In [ ]:
accuracies = []
all_predictions_remapped = []
all_labels_remapped = []
all_predictions_original = []
all_labels_original = []

model.eval()

for ep in range(CONFIG['n_episodes']):
    acc, preds_remapped, labels_remapped, preds_original, labels_original = evaluate_episode(
        model,
        dataset,
        class_to_indices,
        CONFIG['device'],
        n_way=CONFIG['n_way'],
        n_shot=CONFIG['n_shot'],
        n_query=CONFIG['n_query']
    )
    accuracies.append(acc)
    all_predictions_remapped.extend(preds_remapped)
    all_labels_remapped.extend(labels_remapped)
    all_predictions_original.extend(preds_original)
    all_labels_original.extend(labels_original)
    print(f"Episódio {ep+1}: {acc*100:.2f}%")

In [ ]:
mean_acc = np.mean(accuracies)
std_acc  = np.std(accuracies)

print(f"\nAcurácia final: {mean_acc*100:.2f}% ± {std_acc*100:.2f}%")

## 7. Salvamento dos Resultados

In [ ]:
all_predictions_remapped = np.array(all_predictions_remapped)
all_labels_remapped = np.array(all_labels_remapped)
all_predictions_original = np.array(all_predictions_original)
all_labels_original = np.array(all_labels_original)

class_names = dataset.classes

exp_dir = save_fewshot_results(
    experiment_name="deit_prototypical_fewshot",
    model_name=CONFIG['model_name'],
    metric_name="Similaridade Cosine",
    normalization="Normalizacao L2",
    accuracies=accuracies,
    n_way=CONFIG['n_way'],
    n_shot=CONFIG['n_shot'],
    n_query=CONFIG['n_query'],
    n_episodes=CONFIG['n_episodes'],
    device=CONFIG['device'],
    all_predictions=all_predictions_original, 
    all_labels=all_labels_original,             
    class_names=class_names                     
)